In [1]:
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import collections
import pickle
import gensim
import re
import os

In [2]:
with open("words_to_delete.csv") as f:
    words_to_delete = f.readlines()
words_to_delete = [word.strip() for word in words_to_delete]

In [3]:
with open("words_to_len.csv") as f:
    words_to_len = f.readlines()
words_to_len = [word.strip().split(",") for word in words_to_len]
words_to_len = dict([[w2, w1] for w1, w2 in words_to_len])

In [4]:
with open("ingredients_by_recipe.pickle", "rb") as f:
    recipes_ingredients = pickle.load(f)

In [5]:
def remove_paranthesis(i):
    all_paranthesis = re.findall(r"\(.+\)", i)
    for paranthesis in all_paranthesis:
        i = i.replace(paranthesis, "")
    return i.strip()

In [6]:
#remove descriptions
recipes_ingredients = [[i for i in ingredients if i and not i[-1] == ':'] for ingredients in recipes_ingredients]
#remove things in paranthesis
recipes_ingredients = [[remove_paranthesis(i) if "(" in i else i for i in ingredients] for ingredients in recipes_ingredients]
#cahnge / and - and + to space
recipes_ingredients = [[i.replace("+", " ").replace("/", " ").replace("-", " ") for i in ingredients] for ingredients in recipes_ingredients]

In [7]:
all_words = []
[[[all_words.append(w) for w in word_tokenize(i)] for i in ing] for ing in recipes_ingredients]
all_words = collections.Counter(all_words)

In [8]:
words_to_delete = set(words_to_delete).union(set([w for w, c in all_words.items() if c < 5]))

In [9]:
#remove the non-ingredients words from 500 most common words + others
recipes_ingredients = [[[w.replace("*", "").lower().strip() for w in word_tokenize(i) if w not in words_to_delete and len(w) > 2] for i in ing] for ing in recipes_ingredients]

In [10]:
#change words to their lem form
recipes_ingredients = [[[words_to_len[w] if w in words_to_len else w for w in i] for i in ing] for ing in recipes_ingredients]

In [11]:
def unique_ingredients(ingredients):
    all_ingredients = set()
    for ing in ingredients:
        all_ingredients = all_ingredients.union(set(ing))
    return list(all_ingredients)

In [12]:
#make each ingredent only repeat once for each recipe
recipes_ingredients = [unique_ingredients(ingredients) for ingredients in recipes_ingredients]

In [13]:
dic = gensim.corpora.Dictionary(recipes_ingredients)
doc_term = [dic.doc2bow(text) for text in recipes_ingredients]

In [14]:
model = gensim.models.wrappers.ldavowpalwabbit.LdaVowpalWabbit('/usr/bin/vw', 
                                                               corpus=doc_term, 
                                                               id2word=dic,
                                                               num_topics=12,
                                                               passes=20
                                                              )

In [15]:
for topic in model.show_topics(12, formatted=False, num_words=20):
    print(topic)
    print("---")

[(0.28018343, 'limona'), (0.24821432, 'sok'), (0.16453196, 'sladkor'), (0.09117114, 'pomaranča'), (0.07449341, 'voda'), (0.020157397, 'meta'), (0.01865897, 'kis'), (0.017195323, 'limeta'), (0.0101566855, 'avokado'), (0.00746154, 'jogurt'), (0.0072845165, 'gustin'), (0.0069441884, 'čaj'), (0.0057172487, 'beluši'), (0.0054112636, 'bezeg'), (0.0050608884, 'češnja'), (0.0046910737, 'melisa'), (0.0042475485, 'mango'), (0.0040957863, 'jušna-osnova'), (0.0037022945, 'losos'), (0.0022491734, 'konoplja')]
---
[(0.3679371, 'jabolko'), (0.21431293, 'banana'), (0.07428403, 'breskev'), (0.06995175, 'ananas'), (0.062792026, 'jogurt'), (0.037304413, 'kompot'), (0.022667924, 'žele'), (0.022124263, 'kivi'), (0.019102756, 'pesa'), (0.01747859, 'melona'), (0.01267836, 'malina'), (0.011261541, 'mandarina'), (0.009388726, 'alkohol'), (0.008651356, 'lubenica'), (0.0079454165, 'bonbon'), (0.0078351395, 'nektarina'), (0.0071145426, 'viski'), (0.0069790687, 'skorja'), (0.0023367906, 'ovca'), (0.002308046, 'pan